<a href="https://colab.research.google.com/github/Nicolaser1102/Introduccion_to_SQL/blob/main/Ejercicios_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CODING DOJO 
## Fecha creación:31/1/2022
## Fecha de actualización: 31/1/2022
## Esteban Simbaña

#Ejercicios SQL

## *Plantilla para montar el entorno de Python, tomada de Coding Dojo*

In [1]:
# Importamos las librerías que necesitaremos
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# Montamos los datos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# Cargamos tabla 1 (school_details)
school = pd.read_csv('/content/drive/MyDrive/Coding Dojo/Códigos/Track 3/Deberes /Week 4 Track 3/School_Details.csv')
school.head(2)

,DistrictName,EstablishmentCode,EstablishmentName,BRCName,CRCName,SchoolLevel,ManagementName,LocalityHabitation,SubdivisionName,ConstituencyName,...,EstablishedDate,DistrictHQDistance (in Km),CRCDistance (in Km),BACDistance (in Km),IsRecognized,RecognizedBy,ContactPersonName,LowestClass,HighestClass,BoardName
0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,Rhenock,...,28126,58.0,8.0,16.0,Recognized,HRDD,NaN,Class 1,Class 5,NaN
1,East,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Duga Brc,Central Pendam Government Senior Secondary School,SSS,State Govt.,CENTRAL PENDAM,Gangtok,West Pandam,...,15342,50.0,0.0,6.0,Recognized,HRDD,NaN,Class 1,Class 12,C.B.S.E.


In [35]:
# Cargamos tabla 2 (school_facility_details)
facility = pd.read_csv('/content/drive/MyDrive/Coding Dojo/Códigos/Track 3/Deberes /Week 4 Track 3/School_Facility_Details.csv')
facility.head(2)

,EstablishmentCode,EstablishmentName,IsLibraryAvailable,AvailableBooksCount,IsFireExtgAvailable,ExtinguisherCount,HOIRoomAvailable,IsStaffRoomAvailable,NoofStaffRoom,IsLockerForStaffAvailable,IsSportsStoreRoomAvailable
0,S1034,(I.E.C) INDIAN EVANGALISTIC CRUSADE MISSION PR...,Not Available,0,Available,1,Available,Not Available,0,Not Available,Not Available
1,S0516,10TH MILE GOVERNMENT PRIMARY SCHOOL,Not Available,0,Not Available,0,Available,Available,1,Not Available,Not Available


In [7]:
# Creamos sqlite engine
engine2 = create_engine('sqlite:///school.db', echo=True)
# Creamos una conexión a engine

conn = engine2.connect()

In [8]:
# Añadimos la base de datos como una tabla en sqlite
sqlite_table = "school_details"
school.to_sql(sqlite_table, conn, if_exists='fail')

sqlite_table2 = "school_facility_details"
facility.to_sql(sqlite_table2, conn, if_exists='fail')

2022-03-31 23:52:14,949 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("school_details")
2022-03-31 23:52:14,951 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-31 23:52:14,953 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("school_details")
2022-03-31 23:52:14,955 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-31 23:52:14,970 INFO sqlalchemy.engine.Engine 
CREATE TABLE school_details (
	"index" BIGINT, 
	"DistrictName" TEXT, 
	"EstablishmentCode" TEXT, 
	"EstablishmentName" TEXT, 
	"BRCName" TEXT, 
	"CRCName" TEXT, 
	"SchoolLevel" TEXT, 
	"ManagementName" TEXT, 
	"LocalityHabitation" TEXT, 
	"SubdivisionName" TEXT, 
	"ConstituencyName" TEXT, 
	"RevenueBlockName" TEXT, 
	"CivicBodyName" TEXT, 
	"WardName" TEXT, 
	"PostOfficeName" TEXT, 
	"PhoneNo" TEXT, 
	"FaxNo" TEXT, 
	"EmailID" TEXT, 
	"Website" TEXT, 
	"LocatedInRuralOrUrban" TEXT, 
	"DiseCode" TEXT, 
	"EstablishedDate" TEXT, 
	"DistrictHQDistance (in Km)" FLOAT, 
	"CRCDistance (in Km)" FLOAT, 
	"BACDistance (in 

## **Ahora, escribe consultas SQL para responder las siguientespreguntas:**

## Aggregates

### 1. ¿Cuál es el BACDistance promedio (en km)? Renombra la columna resultante como "avg_bac_distancia."

In [25]:
#verificamos cuantas filas tiene la dataframe original
school.shape

(1249, 30)

In [28]:
#obtenemos el promedio manualmente de la dataframe en pandas
(school['BACDistance (in Km)']).sum()/1249

12.651361088871097

In [15]:
# Write your SQL query in this string
sql = """
select AVG("BACDistance (in Km)") as avg_bac_distance

from school_details"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine2)
df.head()

2022-03-31 23:55:10,848 INFO sqlalchemy.engine.Engine 
select AVG("BACDistance (in Km)") as avg_bac_distance

from school_details
2022-03-31 23:55:10,851 INFO sqlalchemy.engine.Engine [raw sql] ()


,avg_bac_distance
0,12.681822


Se verifica que nuestra primera consulta sql está bien hecha comparando el resultado del promedio obtenido manualmente **12.65**, con el obtenido mediante la consulta sql **12.68**

## GROUP BY

### 2. ¿Cuál es el BACDistance promedio (en km) según nivel educativo del colegio?  Ordena los resultados de promedio mayor a promedio menor.

In [18]:
#observamos cuantos niveles de educación existen en la data frame original
school["SchoolLevel"].unique()

array(['PS', 'SSS', 'SS', 'JHS', 'LPS', nan], dtype=object)

In [21]:
# Write your SQL query in this string
sql = """

SELECT "SchoolLevel", AVG("BACDistance (in Km)") AS avg_BACDistanceProm
FROM school_details
GROUP BY "SchoolLevel"
ORDER BY "avg_BACDistanceProm" DESC
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine2)
df

2022-04-01 00:03:14,186 INFO sqlalchemy.engine.Engine 

SELECT "SchoolLevel", AVG("BACDistance (in Km)") AS avg_BACDistanceProm
FROM school_details
GROUP BY "SchoolLevel"
ORDER BY "avg_BACDistanceProm" DESC

2022-04-01 00:03:14,192 INFO sqlalchemy.engine.Engine [raw sql] ()


,SchoolLevel,avg_BACDistanceProm
0,JHS,13.692647
1,PS,12.830787
2,SS,12.630872
3,SSS,10.863095
4,LPS,5.368421
5,None,NaN


## Having

### 3. Repite la consulta anterior, pero solo muestra los resultados donde la distancia promedio sea menor a 10 km.

In [29]:
# Write your SQL query in this string
sql = """

SELECT "SchoolLevel", AVG("BACDistance (in Km)") AS avg_BACDistanceProm
FROM school_details
GROUP BY "SchoolLevel"

HAVING avg_BACDistanceProm < 10

ORDER BY "avg_BACDistanceProm" DESC



"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine2)
df.head()

2022-04-01 00:08:44,817 INFO sqlalchemy.engine.Engine 

SELECT "SchoolLevel", AVG("BACDistance (in Km)") AS avg_BACDistanceProm
FROM school_details
GROUP BY "SchoolLevel"

HAVING avg_BACDistanceProm < 10

ORDER BY "avg_BACDistanceProm" DESC




2022-04-01 00:08:44,822 INFO sqlalchemy.engine.Engine [raw sql] ()


,SchoolLevel,avg_BACDistanceProm
0,LPS,5.368421


## Joins

### 4. Une las dos tablas (detalles de colegios y detalles de instalaciones de los colegios). Comienza mostrando todas las columnas, pero limita las filas resultantes a solo 2.

In [60]:
# Write your SQL query in this string
sql = """

SELECT *
FROM
   school_details
INNER JOIN
   school_facility_details
ON school_details.EstablishmentCode = school_facility_details.EstablishmentCode

"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine2)
df.head(2)

2022-04-01 00:51:07,754 INFO sqlalchemy.engine.Engine 

SELECT *
FROM
   school_details
INNER JOIN
   school_facility_details
ON school_details.EstablishmentCode = school_facility_details.EstablishmentCode


2022-04-01 00:51:07,763 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,DistrictName,EstablishmentCode,EstablishmentName,BRCName,CRCName,SchoolLevel,ManagementName,LocalityHabitation,SubdivisionName,...,EstablishmentName,IsLibraryAvailable,AvailableBooksCount,IsFireExtgAvailable,ExtinguisherCount,HOIRoomAvailable,IsStaffRoomAvailable,NoofStaffRoom,IsLockerForStaffAvailable,IsSportsStoreRoomAvailable
0,0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,...,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Available,100,Available,2,Available,Available,1,Not Available,Not Available
1,1,East,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Duga Brc,Central Pendam Government Senior Secondary School,SSS,State Govt.,CENTRAL PENDAM,Gangtok,...,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Available,0,Available,5,Available,Available,2,Not Available,Available


Plantilla para realizar uniones en SQL inspirada de:
https://desarrolloweb.com/articulos/2355.php

### 5. Ahora, solo selecciona colegios donde el nivel de este sea Escuela Primaria y que no tengan biblioteca.

Escuela Primaria = "PS"

In [62]:
facility["IsLibraryAvailable"].unique()

array(['Not Available', 'Available'], dtype=object)

In [68]:
# Write your SQL query in this string
sql = """

SELECT *
FROM
   school_details
INNER JOIN
   school_facility_details
ON school_details.EstablishmentCode = school_facility_details.EstablishmentCode

WHERE SchoolLevel = "PS" AND IsLibraryAvailable = "Available"

ORDER BY ciudad DESC
 
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine2)
df.head(2)


2022-04-01 00:54:37,750 INFO sqlalchemy.engine.Engine 

SELECT *
FROM
   school_details
INNER JOIN
   school_facility_details
ON school_details.EstablishmentCode = school_facility_details.EstablishmentCode

WHERE SchoolLevel = "PS" AND IsLibraryAvailable = "Available"
 

2022-04-01 00:54:37,754 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,DistrictName,EstablishmentCode,EstablishmentName,BRCName,CRCName,SchoolLevel,ManagementName,LocalityHabitation,SubdivisionName,...,EstablishmentName,IsLibraryAvailable,AvailableBooksCount,IsFireExtgAvailable,ExtinguisherCount,HOIRoomAvailable,IsStaffRoomAvailable,NoofStaffRoom,IsLockerForStaffAvailable,IsSportsStoreRoomAvailable
0,0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,...,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Available,100,Available,2,Available,Available,1,Not Available,Not Available
1,6,East,S0870,"ETERNAL ENGLISH SCHOOL, WEST PENDAM",Duga Brc,Central Pendam Government Senior Secondary School,PS,Private (Unaided),SAKHU WEST PENDAM,Gangtok,...,"ETERNAL ENGLISH SCHOOL, WEST PENDAM",Available,0,Available,1,Not Available,Not Available,0,Not Available,Not Available


### 6. Toma la consulta anterior, pero ordena los resultados en orden alfabético por nombre.

In [70]:
# Write your SQL query in this string
sql = """

SELECT *
FROM
   school_details
INNER JOIN
   school_facility_details
ON school_details.EstablishmentCode = school_facility_details.EstablishmentCode

WHERE SchoolLevel = "PS" AND IsLibraryAvailable = "Available"

ORDER BY EstablishmentName 

"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine2)
df.head()

2022-04-01 00:58:27,845 INFO sqlalchemy.engine.Engine 

SELECT *
FROM
   school_details
INNER JOIN
   school_facility_details
ON school_details.EstablishmentCode = school_facility_details.EstablishmentCode

WHERE SchoolLevel = "PS" AND IsLibraryAvailable = "Available"

ORDER BY EstablishmentName 


2022-04-01 00:58:27,849 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,DistrictName,EstablishmentCode,EstablishmentName,BRCName,CRCName,SchoolLevel,ManagementName,LocalityHabitation,SubdivisionName,...,EstablishmentName,IsLibraryAvailable,AvailableBooksCount,IsFireExtgAvailable,ExtinguisherCount,HOIRoomAvailable,IsStaffRoomAvailable,NoofStaffRoom,IsLockerForStaffAvailable,IsSportsStoreRoomAvailable
0,345,East,S0178,AGRIGAON GOVERNMENT PRIMARY SCHOOL,Regu Brc,Dalapchen Government Secondary School,PS,State Govt.,AGRIGAON BUSTY,Rongli,...,AGRIGAON GOVERNMENT PRIMARY SCHOOL,Available,60,Available,2,Not Available,Available,1,Not Available,Not Available
1,385,East,S0146,ANKUCHEN GOVERNMENT PRIMARY SCHOOL,Pakyong Brc,Linkey Government Secondary School,PS,State Govt.,ANKUCHEN BUSTY,Pakyong,...,ANKUCHEN GOVERNMENT PRIMARY SCHOOL,Available,300,Available,2,Not Available,Available,1,Not Available,Not Available
2,143,East,S0187,BADONG THANGSING GOVERNMENT PRIMARY SCHOOL,Khamdong Brc,Khamdong Government Senior Secondary School,PS,State Govt.,BADONG THANGSING,Gangtok,...,BADONG THANGSING GOVERNMENT PRIMARY SCHOOL,Available,13,Available,2,Not Available,Available,1,Not Available,Not Available
3,457,North,S0067,BAKCHA GOVERNMENT PRIMARY SCHOOL,Kabi Brc,Kalzang Gyatso Government senior Secondary School,PS,State Govt.,SAMDONGCHEN,Mangan,...,BAKCHA GOVERNMENT PRIMARY SCHOOL,Available,6,Available,2,Not Available,Available,1,Not Available,Not Available
4,1149,West,S1217,"BAL JYOTI ACADEMY, TIMBURBONG",Soreng Brc,Soreng Government Senior Secondary School,PS,Private (Unaided),TIMBURBONG,Soreng,...,"BAL JYOTI ACADEMY, TIMBURBONG",Available,0,Available,0,Not Available,Available,1,Available,Available


In [ ]:
# Close the database connection
conn.close()